# GloVe-BiLSTM wit Cross Validation

This Notebook presents the training of the GloVe-BiLSTM model with the use of k-fold cross validation.

# Load Data

In [8]:
import torch
import random
import time
from torchtext.legacy import data
from sklearn.model_selection import KFold
import numpy as np
import torch.optim as optim

# This class loads the dataset
class load_data(object):

    # Initialize the parameters
    def __init__(self, SEED=1234):
        torch.manual_seed(SEED)
        torch.cuda.manual_seed(SEED)
        torch.backends.cudnn.enabled = False 
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True

        # Create Field objects
        # use Spacy tokenizer
        # specify English language
        # need lenghts for padded sequences
        PHRASE = data.Field(tokenize = 'spacy',
                        tokenizer_language = 'en_core_web_sm',
                        include_lengths = True)
        SENTIMENT = data.LabelField()

        # Create tuples representing the columns
        fields = [
                (None, None), # ignore PhraseId column
                (None, None), # ignore SentenceId column
                ('Phrase', PHRASE),
                ('Sentiment', SENTIMENT)
        ]

        # Load the dataset in tsv format
        self.train_ds, self.test_ds = data.TabularDataset.splits(
                        path = './data',
                        train = 'train.tsv',
                        test = 'test_mapped.tsv',
                        format = 'tsv',
                        fields = fields,
                        skip_header = True
        )
        
        self.SEED = SEED

    # Shhuffle the data using k-fold Cross Validation
    def get_fold_data(self, num_folds):
        
        # Create Field objects
        PHRASE = data.Field(tokenize = 'spacy',
                        tokenizer_language = 'en_core_web_sm',
                        include_lengths = True)
        SENTIMENT = data.LabelField()

        # Create tuples representing the columns
        fields = [
                (None, None), # ignore PhraseId column
                (None, None), # ignore SentenceId column
                ('Phrase', PHRASE),
                ('Sentiment', SENTIMENT)
        ]
        
        # k-Fold Cross Validation
        kf = KFold(n_splits=num_folds, shuffle=True, random_state = random.seed(self.SEED))
        train_data_arr = np.array(self.train_ds.examples)

        for train_index, val_index in kf.split(train_data_arr):
            yield(
                PHRASE,
                SENTIMENT,
                data.Dataset(train_data_arr[train_index], fields=fields),
                data.Dataset(train_data_arr[val_index], fields=fields),
            )
    
    def get_test_data(self):
        return self.test_ds

# Model Definition

In [9]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        #Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        #LSTM layer
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        #Fully-Connected Linear Layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #Dropout functionality for reqularizotion
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack embedding sequence to process only the non-padded elements of our sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence; no used in our case
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [10]:
def categorical_accuracy(preds, y):
    #Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

# Train and Evaluation methods

In [11]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.Phrase
        
        predictions = model(text, text_lengths)
        
        loss = criterion(predictions, batch.Sentiment)
        
        acc = categorical_accuracy(predictions, batch.Sentiment)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [12]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.Phrase
            
            predictions = model(text, text_lengths)
            
            loss = criterion(predictions, batch.Sentiment)
            
            acc = categorical_accuracy(predictions, batch.Sentiment)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [13]:
def epoch_time(start_time, end_time):
    # calculate the duration of the training time for one epoch
    
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Model Training

In [14]:
data_generator = load_data()
_history = []
device = None
model = None
criterion = None
fold_index = 0

N_FOLDS = 5

for PHRASE, SENTIMENT, train_ds, val_ds in data_generator.get_fold_data(num_folds=N_FOLDS):
    
    print("***** Running Training *****")
    print(f"Now fold: {fold_index + 1} / {N_FOLDS}")

    # Build vocabulary (using Glove pre-trained word embeddings, dimensinality = 100)
    PHRASE.build_vocab(train_ds,
                        vectors = "glove.6B.100d", 
                        unk_init = torch.Tensor.normal_)

    sentiment_labels = np.array(['0', '1', '2', '3', '4'])

    #SENTIMENT.build_vocab(train_ds) # old version
    SENTIMENT.build_vocab(sentiment_labels) # Custom indexing is necessary for correct mapping; {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4}

    # Initialize model
    INPUT_DIM = len(PHRASE.vocab)
    EMBEDDING_DIM = 100
    HIDDEN_DIM = 256
    OUTPUT_DIM = len(SENTIMENT.vocab)
    N_LAYERS = 2
    BIDIRECTIONAL = True
    DROPOUT = 0.5
    PAD_IDX = PHRASE.vocab.stoi[PHRASE.pad_token]

    model = LSTM(INPUT_DIM, 
                EMBEDDING_DIM, 
                HIDDEN_DIM, 
                OUTPUT_DIM, 
                N_LAYERS, 
                BIDIRECTIONAL, 
                DROPOUT, 
                PAD_IDX)

    # replace the initial weights of the embedding layer with the pre-trained embeddings
    pretrained_embeddings = PHRASE.vocab.vectors
    model.embedding.weight.data.copy_(pretrained_embeddings)

    UNK_IDX = PHRASE.vocab.stoi[PHRASE.unk_token]
    #  initialize <unk> and <pad> tokens to all zeros
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

    # Init optimizer and loss function
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()

    # Only necessary for GPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = model.to(device)
    criterion = criterion.to(device)

    BATCH_SIZE = 32

    # Creating the iterators
    train_iterator = data.Iterator(train_ds, 
                                batch_size=BATCH_SIZE, 
                                sort = False,
                                sort_key = lambda x: len(x.Phrase),
                                sort_within_batch = True,
                                device = device)
    valid_iterator = data.Iterator(val_ds, 
                                batch_size=BATCH_SIZE, 
                                sort = False,
                                sort_key = lambda x: len(x.Phrase),
                                sort_within_batch = True,
                                device = device) 

    N_EPOCHS = 10

    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'model_kFold.pt')

        _history.append([valid_loss, valid_acc])
    
    fold_index += 1
    
_history = np.asarray(_history)
loss = np.mean(_history[:, 0])
acc = np.mean(_history[:, 1])
    
print('***** Cross Validation Result *****')
print(f'LOSS: {loss}, ACC: {acc}') 

***** Running Training *****
Now fold: 1 / 5


KeyboardInterrupt: 